<a href="https://colab.research.google.com/github/PaulinaTarkowsk/sql_recap/blob/main/dash_cross_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cross filtering w dash

### Przygotowanie notatnika

Zaczynamy od zainstalowania i importu potrzebnych bibliotek.

In [28]:
!pip install dash

In [29]:
import dash
import json
import pandas as pd
import numpy as np
import plotly.express as px
import threading
from dash import dcc, html, Input, Output
from google.colab import output, drive

In [30]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Tworzenie wizualizacji

#### Rozwiązania wspierające

Przygotowanie potrzebnych danych

In [31]:
population = pd.read_csv('/content/drive/My Drive/Dash/Dash_crossfiltering/population.csv')
new_books = pd.read_csv("/content/drive/My Drive/Dash/Dash_crossfiltering/new-books-per-million.csv")
new_books = new_books[(new_books["Year"]>=1940) & (new_books["Year"]<=1996)]
selected_books = new_books[new_books["Year"]==1996][["Entity", "Book titles per capita (Fink-Jensen 2015)"]]
selected_pops = population[population["Year"]==1996][["Entity", "Population (historical estimates)"]]
data = pd.merge(selected_books, selected_pops, how="inner", on="Entity")

Stworzenie wykresu wykorzystywanego w aplikacji

In [32]:
scatter_figure = px.scatter(data_frame=data,
                 x="Book titles per capita (Fink-Jensen 2015)",
                 y="Population (historical estimates)",
                 text="Entity",
                 log_x=True,
                 log_y=True,
                 template="none",
                 title="Country population vs number of new book titles per capita in 1996")
scatter_figure.update_traces(textposition="bottom right")

Poniżej mamy kod pierwszej aplikacji wykorzystanej w prezentacji **dash Crossfiltering**.

In [37]:
app = dash.Dash()
app.layout = html.Div(
        children=[
                  html.P(id='div_value'),
                  dcc.Graph(id='scatter', figure=scatter_figure),
                  ],
)

@app.callback(
    Output('div_value', 'children'),
    Input('scatter', 'clickData')
)
def show_me(data):
   return json.dumps(data)

Poniżej mamy kod drugiej aplikacji wykorzystanej w prezentacji **dash Crossfiltering**.

In [39]:
app = dash.Dash()
app.layout = html.Div(
    children=[
              dcc.Graph(
                  id='scatter',
                  figure=scatter_figure
                  ),
              dcc.Graph(id='line')
              ],
)

@app.callback(
    Output('line', 'figure'),
    Input('scatter','hoverData')
)
def build_line_chart(country):

  selected_country = country["points"][0]["text"]
  selected_data = new_books[new_books["Entity"]==selected_country]

  fig = px.line(data_frame=selected_data,
                x="Year",
                y="Book titles per capita (Fink-Jensen 2015)",
                template="none",
                title="Book titles per capita in %s for years 1940-1996" % selected_country
                )
  return fig

### Wyświetlenie wizualizacji

Aby zbudować wizualizację z wykorzystaniem powyższego kodu, potrzebujemy jeszcze uruchomić serwer i przejść na stronę aplikacji.


In [40]:
thread = threading.Thread(target=app.run_server)
thread.start()
output.serve_kernel_port_as_window(port=8050)

<IPython.core.display.Javascript object>

In [38]:
thread = threading.Thread(target=app.run_server)
thread.start()
output.serve_kernel_port_as_window(port=8050)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>